# Whereabouts: calculation of speed, accuracy and comparison with other geocoders

06 09 24

## Description
This notebook contains code for producing the accuracy and speed results for the whereabouts package (https://www.github.com/ajl2718/whereabouts) and comparing with three other geocoders: Google, MapBox and Nominatim.


## Test datasets
There are three datasets used for testing, two for accuracy and one for speed:
- Locations of Guzman and Gomez stores in Australia
- Random residential locations from parts of Australia
- Licensed venues in Victoria, Australia

These datasets are available here: https://github.com/ajl2718/python_learning/whereabouts_testing/

## Methodology
Accuracy is assessed at four levels of geographic granularity:
- Apartment number: All components of the address must be correct including level number, building number, shop number where these appear
- House number: The main street number has to be correct
- Street name: The address is correct up to the street name
- Suburb name: The correct suburb of the location is found

For each level, a score of 1 is given if the address matches and 0 otherwise. Total accuracy (number of correct / number of addresses) is then calculated for each level

In [7]:
from time import time, sleep
import requests

import pandas as pd
import numpy as np

import math

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

In [8]:
sns.set_style("whitegrid")

## Data sources

In [9]:
# load Guzman y Gomez
df = pd.read_csv('gyg_210624_geocoded.csv')
addresses_gyg = df['address'].values

In [10]:
# load residential addresses
df = pd.read_csv('rea_130824.csv', sep='\t')
addresses_rea = df['address'].sample(128, random_state=42).values

## Geocoding: whereabouts

In [14]:
from whereabouts.Matcher import Matcher
from whereabouts.MatcherPipeline import MatcherPipeline

In [15]:
matcher1 = Matcher('au_all_lg', how='standard')
matcher2 = Matcher('au_all_lg', how='trigram')

pipeline = MatcherPipeline([matcher1, matcher2])

In [18]:
list(addresses_gyg)

['501 COWPASTURE RD HOXTON PARK 2171 NSW',
 'SHOP 1 57 OLD NORTHERN RD ALBANY CREEK 4035 QLD',
 '175 KING STREET NEWTOWN 2042 NSW',
 '1A BRONTE RD BONDI JUNCTION SYDNEY 2022 NSW',
 'BAYSWATER ROAD CNR PENNYS LANE KINGS CROSS 2011 NSW',
 'SHOP 403 LEVEL 4 264 278 GEORGE STREET SYDNEY 2000 NSW',
 'SHOP 14 LEVEL 6 MLC CENTRE FOOD COURT MARTIN PLACE CASTLEREAGH STREET SYDNEY 2000 NSW',
 'SHOP 5008 LEVEL 5 WESTFIELD SYDNEY FOOD COURT 188 PITT STREET SYDNEY 2000 NSW',
 'SHOP 2 77 WILLOUGHBY ROAD CRN HOLTERMAN STREET CROWS NEST 2065 NSW',
 'SHOP 6013 LEVEL 6 WESTFIELD BONDI JUNCTION 500 OXFORD STREET BONDI JUNCTION 2022 NSW',
 'SHOP EG07 07B BUNDA ST CANBERRA CENTRE SHOPPING CENTRE CANBERRA CITY 2601 ACT',
 '86 BEAUMONT STREET HAMILTON 2303 NSW',
 'SHOP 14 MANLY WHARF EAST ESPLANADE MANLY 2095 NSW',
 'SHOP 10 31 WORLD SQUARE SHOPPING CENTRE 680 GEORGE ST SYDNEY 2000 NSW',
 'SHOP 73 RHODES WATERSIDE SHOPPING CENTRE 1 RIDER BLVD RHODES 2138 NSW',
 'UNIVERSITY TERRACES LOWER KENSINGTON CAMPUS HI

In [20]:
# use matcher pipeline
results_gyg = matcher1.geocode(list(addresses_gyg))

In [22]:
df_results = pd.DataFrame(data=results_gyg)
df_results.to_csv('gyg_experiment_whereabouts_030924.csv', index=False)

In [24]:
results_rea = pipeline.geocode(list(addresses_rea))

In [25]:
df_results = pd.DataFrame(data=results_rea)
df_results.to_csv('rea_experiment_whereabouts_030924.csv', index=False)

## Nominatim

In [27]:
headers = {
    "User-Agent": "GeoTesting/1.0 (ajlee3141@gmail.com)"
}


In [28]:
# add country name to addresses
addresses_gyg = [address + ' AUSTRALIA' for address in addresses_gyg]
addresses_rea = [address + ' AUSTRALIA' for address in addresses_rea]

In [29]:
all_results = []

for address in addresses_rea:
    sleep(1)
    print(f'Geocoding {address}')
    address_new = address.replace(' ', '+')
    url_base = 'https://nominatim.openstreetmap.org/search'
    params = {'q': address_new,
              'format': 'json',
              'addressdetails': 1}
    page = requests.get(url_base, params=params, headers=headers)
    results = page.json()
    if isinstance(results, list):
        if len(results) > 0:
            results_new = results[0]
            all_results.append(results_new)
        else:
            all_results.append({})
    elif isinstance(results, dict):
        results_new = results
        all_results.append(results_new)
    else:
        all_results.append({})
    

Geocoding 5 Dylan Crt, Stanthorpe, Qld 4380 AUSTRALIA
Geocoding 23 Northbridge Road, Highton, Vic 3216 AUSTRALIA
Geocoding 62 Radstock Avenue, Highton, Vic 3216 AUSTRALIA
Geocoding Lot 3 Texas Road, Stanthorpe, Qld 4380 AUSTRALIA
Geocoding 44 Verner Street, South Geelong, Vic 3220 AUSTRALIA
Geocoding 616 Woodhill Mountain Road, Woodhill, NSW 2535 AUSTRALIA
Geocoding 501/567 Pacific Hwy, St Leonards, NSW 2065 AUSTRALIA
Geocoding Unit 3/16 Poinciana Bvd, Cardwell, Qld 4849 AUSTRALIA
Geocoding Lot 3, 232 Sugarloaf Road, Stanthorpe, Qld 4380 AUSTRALIA
Geocoding 10/57-59 Lachlan Street, Warwick Farm, NSW 2170 AUSTRALIA
Geocoding 10 Ilaroo Crescent, Warana, Qld 4575 AUSTRALIA
Geocoding 801/51 Chandos Street, St Leonards, NSW 2065 AUSTRALIA
Geocoding 75 Reema Boulevard, Endeavour Hills, Vic 3802 AUSTRALIA
Geocoding 29 Jamieson St, Cardwell, Qld 4849 AUSTRALIA
Geocoding 31/89-91 Hughes Street, Cabramatta, NSW 2166 AUSTRALIA
Geocoding 2/11 Cedarville Close, Highton, Vic 3216 AUSTRALIA
Geocoding

In [21]:
# write the results
(
    pd.DataFrame(all_results)
    .assign(input_address=addresses_rea)
    .to_csv('rea_experiments_nominatim_030924.csv', index=False)
)

## Googlemaps API

In [90]:
api_key = '' # set this

In [ ]:
url_base = 'https://maps.googleapis.com/maps/api/geocode/json'

In [92]:
all_results = []

dfs = []

for address in addresses_rea:
    print(f'Geocoding {address}')
    address_new = address.replace(' ', '+')
    params = {'address': address_new, 'key': api_key}
    page = requests.get(url=url_base, params=params)
    df_temp = (
        pd.DataFrame(page.json()['results'][0]['address_components'])[['long_name']]
        .T
        .reset_index()
        .rename(columns={0: 'number', 1: 'street', 2: 'name', 3: 'suburb', 4: 'state', 5: 'country', 6: 'postcode'})
    )
    dfs.append(df_temp)

Geocoding 5 Dylan Crt, Stanthorpe, Qld 4380
Geocoding 23 Northbridge Road, Highton, Vic 3216
Geocoding 62 Radstock Avenue, Highton, Vic 3216
Geocoding Lot 3 Texas Road, Stanthorpe, Qld 4380
Geocoding 44 Verner Street, South Geelong, Vic 3220
Geocoding 616 Woodhill Mountain Road, Woodhill, NSW 2535
Geocoding 501/567 Pacific Hwy, St Leonards, NSW 2065
Geocoding Unit 3/16 Poinciana Bvd, Cardwell, Qld 4849
Geocoding Lot 3, 232 Sugarloaf Road, Stanthorpe, Qld 4380
Geocoding 10/57-59 Lachlan Street, Warwick Farm, NSW 2170
Geocoding 10 Ilaroo Crescent, Warana, Qld 4575
Geocoding 801/51 Chandos Street, St Leonards, NSW 2065
Geocoding 75 Reema Boulevard, Endeavour Hills, Vic 3802
Geocoding 29 Jamieson St, Cardwell, Qld 4849
Geocoding 31/89-91 Hughes Street, Cabramatta, NSW 2166
Geocoding 2/11 Cedarville Close, Highton, Vic 3216
Geocoding 1201/29 HUNTER STREET, Parramatta, NSW 2150
Geocoding 13 Mainsbridge Avenue, Liverpool, NSW 2170
Geocoding 1307 Midge Point Road, Midge Point, Qld 4799
Geocodi

In [1]:
# write the results
(
    pd.concat(dfs)
    .assign(input_address=addresses_rea)
    .to_csv('/rea_experiments_google_030924.csv', index=False)
)

## Mapbox


In [77]:
access_token = '' # set this

In [78]:
url_base = 'https://api.mapbox.com/search/geocode/v6/forward'
url_full = f'{url_base}?q={address}&access_token='+access_token

In [85]:
all_results = []

dfs = []

for address in addresses_rea:
    print(f'Geocoding {address}')
    address_new = address.replace(' ', '+')
    url_full = f'{url_base}?q={address_new}&access_token='+access_token
    page = requests.get(url=url_full)
    address_result = pd.DataFrame(page.json()['features'][0])['properties']
    dfs.append(address_result)

Geocoding 5 Dylan Crt, Stanthorpe, Qld 4380
Geocoding 23 Northbridge Road, Highton, Vic 3216
Geocoding 62 Radstock Avenue, Highton, Vic 3216
Geocoding Lot 3 Texas Road, Stanthorpe, Qld 4380
Geocoding 44 Verner Street, South Geelong, Vic 3220
Geocoding 616 Woodhill Mountain Road, Woodhill, NSW 2535
Geocoding 501/567 Pacific Hwy, St Leonards, NSW 2065
Geocoding Unit 3/16 Poinciana Bvd, Cardwell, Qld 4849
Geocoding Lot 3, 232 Sugarloaf Road, Stanthorpe, Qld 4380
Geocoding 10/57-59 Lachlan Street, Warwick Farm, NSW 2170
Geocoding 10 Ilaroo Crescent, Warana, Qld 4575
Geocoding 801/51 Chandos Street, St Leonards, NSW 2065
Geocoding 75 Reema Boulevard, Endeavour Hills, Vic 3802
Geocoding 29 Jamieson St, Cardwell, Qld 4849
Geocoding 31/89-91 Hughes Street, Cabramatta, NSW 2166
Geocoding 2/11 Cedarville Close, Highton, Vic 3216
Geocoding 1201/29 HUNTER STREET, Parramatta, NSW 2150
Geocoding 13 Mainsbridge Avenue, Liverpool, NSW 2170
Geocoding 1307 Midge Point Road, Midge Point, Qld 4799
Geocodi

In [86]:
# write the data
(
    pd.DataFrame(dfs)
    .assign(input_address=addresses_rea)
    .to_csv('rea_experiments_mapbox_030924.csv', index=False)
)

## Overall accuracy results

These are found by manually reviewing the outputs from each of the geocoders. See the labelled outputs for the original data used to calculate these values.

In [ ]:
overall_results_data = {'google': [0.675257732, 0.8969072165, 0.9742268041, 0.9896907216, 
                                   0.984375, 1, 1, 1],
                        'whereabouts': [0.6701030928, 0.9536082474, 0.9484536082, 0.9536082474,
                                        0.90625, 0.984375, 0.984375, 1],
                        'mapbox': [0.4639175258, 0.7731958763, 0.8298969072, 0.8453608247, 
                                   0.5234375, 0.578125, 0.984375, 0.984375],
                        'nominatim': [0.1494845361, 0.1494845361, 0.2164948454, 0.2164948454, 
                                      0.1171875, 0.1171875, 0.7265625, 0.71875]}

overall_results_labels1 = ['apartment', 'house', 'street', 'suburb'] + ['apartment', 'house', 'street', 'suburb']
overall_results_labels2 = ['retail', 'retail', 'retail', 'retail'] + ['residential', 'residential', 'residential', 'residential']

df_overall_results = (
    pd.DataFrame(data=overall_results_data)
    .assign(address_level=overall_results_labels1,
            location_type=overall_results_labels2)
    .melt(id_vars=['address_level', 'location_type'], 
          value_vars=['google', 'whereabouts', 'mapbox', 'nominatim'])
)

custom_palette = {
    'whereabouts': '#c34a36',   
    'google': '#b0a8b9', 
    'mapbox': '#bea6a0', 
    'nominatim': '#b9b2d9'
}

In [ ]:
plt.figure(figsize=(8, 6))
sns.barplot(df_overall_results.query('location_type=="retail"'),
            x='address_level',
            y='value',
            hue='variable', 
            palette=custom_palette)
plt.xlabel('Geographic granularity')
plt.ylabel('Accuracy')
plt.title('Accuracy comparison for retail dataset')
plt.legend(loc='lower right')
plt.savefig('geocoder_comparison_retail_050924.png')

## Speed comparison

The %%timeit function is used which carries out 7 runs. The average speed is then calculated. 

These calculations are done for two databases:
- The large Victoria database: this handles a broader range of spelling errors
- The small Australian database: country wide but with less tolerance for spelling errors

In [67]:
# load the data
df = pd.read_excel('liquor.xlsx', skiprows=3)

In [68]:
# calculate the full address
df = (
    df
    .query('Postcode.isnull() == False')
    .assign(Postcode=lambda df_: df_.Postcode.astype(int))
    .loc[:, ['Address', 'Suburb', 'Postcode']]
    .assign(full_address=lambda df_: df_.Address + ' ' + df_.Suburb + ' ' + df_.Postcode.astype(str))
)

In [69]:
# take a random sample of 8192 addresses
addresses = df.full_address.sample(2**13, random_state=42).values

In [76]:
matcher_vic = Matcher('au_all_sm')

In [77]:
%%timeit
results = matcher_vic.geocode(addresses)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

16.9 s ± 40.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
# au_vic_lg
8192 / 15.1

542.5165562913908

In [78]:
# au_all_sm
8192 / 16.9

484.7337278106509